Necessary Libraries

In [ ]:
import pandas as pd
from math import log2

Loading Dataset from Kaggle (Link given below)

https://www.kaggle.com/datasets/uciml/mushroom-classification?resource=download

In [ ]:
df = pd.read_csv('mushrooms.csv')
df.head(10)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


Entropy and Information Gain Functions

In [ ]:
def entropy_calculator(df,target):
    total = len(df)
    classes = df[target].unique()
    entropy = 0
    for c in classes:
        t_class = df[df[target] == c]
        class_total = len(t_class)
        p_class = class_total / total
        entropy -= (p_class*log2(p_class))
    return entropy

def Information_Gain(df,target,column):
    N = len(df)
    Hs = entropy_calculator(df,target)
    column_class = df[column].unique()
    column_class_entropy_sum = 0
    for c in column_class:
        df_column_class = df[df[column] == c]
        Px = len(df_column_class) / N
        Hx = entropy_calculator(df_column_class,target)
        column_class_entropy_sum += (Px*Hx)
    IG = Hs - column_class_entropy_sum
    return IG

def Maximal_Information_Gain(df,target):
    columns = list(df.columns)
    Max_IG = -1
    Split_Column = None
    for c in columns:
        if c == target:
            continue
        ig_column = Information_Gain(df,target,c)
        if ig_column > Max_IG:
            Max_IG = ig_column
            Split_Column = c
    return Max_IG, Split_Column

def singular_column_predict(df,target):
    df = df[[target]]
    frequency_map = {}
    for i in range(0,len(df)):
        try:
            frequency_map[df[i,0]] += 1
        except:
            frequency_map[df[i,0]] = 1
    prediction = None
    max_frequency = 0
    for key in frequency_map:
        if(frequency_map[key] > max_frequency):
            max_frequency = frequency_map
            prediction = key
    return prediction

Tree Data Structure, Construction and Prediction

In [ ]:
class TreeNode:
    def __init__(self):
        self.NextColumn = None
        self.children = {}
        self.leaf = False
        self.entropy = None
        self.prediction = None
        self.information_gain = None

def predict(node,data):
    if node.leaf == True:
        return node.prediction
    return predict(node.children[data[node.NextColumn]],data)

def DecisionTree_Constructor(node,df,target):
    existing_columns = list(df.columns)
    node.entropy = entropy_calculator(df,target)
    print("Entropy at Current Node: " + str(node.entropy))
    if len(existing_columns) == 2:
        node.leaf = True
        node.prediction = singular_column_predict(df,target)
        return
    node.information_gain, node.NextColumn = Maximal_Information_Gain(df,target)
    print("Information Gain is: " + str(node.information_gain) + ", by splitting with " + str(node.NextColumn))
    classes = df[node.NextColumn].unique()
    existing_columns.remove(node.NextColumn)
    for c in classes:
        filtered_df = df[df[node.NextColumn] == c]
        unique_targets = filtered_df[target].unique()
        if len(unique_targets) == 1:
            node.children[c] = TreeNode()
            node.children[c].leaf = True
            node.children[c].prediction = unique_targets[0]
        else:
            node.children[c] = TreeNode()
            filtered_df = filtered_df[existing_columns]
            DecisionTree_Constructor(node.children[c],filtered_df,target)
    return

def Construct_DecisionTree(df,target):
    TreeHead = TreeNode()
    DecisionTree_Constructor(TreeHead,df,target)
    return TreeHead

Shuffling the dataframe and test train splitting

In [ ]:
df_shuffled = df.sample(frac=1, random_state=13)
split_index = int(0.7 * len(df_shuffled))
df_train = df_shuffled.iloc[:split_index]
df_test = df_shuffled.iloc[split_index:]

Training the Model

In [ ]:
DecisionTree = Construct_DecisionTree(df_train,'class')

Entropy at Current Node: 0.9994287474837168
Information Gain is: 0.910186529110717, by splitting with odor
Entropy at Current Node: 0.20890541526096207
Information Gain is: 0.13833934417231253, by splitting with spore-print-color
Entropy at Current Node: 0.39954542348328603
Information Gain is: 0.2902549314245491, by splitting with habitat
Entropy at Current Node: 0.8582307926411411
Information Gain is: 0.8582307926411411, by splitting with cap-color
Entropy at Current Node: 0.5159469300074474
Information Gain is: 0.5159469300074474, by splitting with gill-size


Prediction

In [ ]:
Accuracy = 0
predictions = []
for i in range(0,len(df_test)):
    row_data = df_test.iloc[i].to_dict()
    if row_data['class'] == predict(DecisionTree,row_data):
        Accuracy += 1
    predictions.append(predict(DecisionTree,row_data))

print(predictions)
print("Accuracy: " + str(Accuracy / len(df_test)))

['e', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'p', 'p', 'e', 'e', 'p', 'p', 'e', 'p', 'p', 'p', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'e', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'p', 'e', 'p', 'p', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'p', 'p', 'e', 'p', 'p', 'p', 'e', 'p', 'p', 'e', 'p', 'p', 'e', 'e', 'e', 'e', 'e', 'p', 'e', 'p', 'p', 'e', 'e', 'p', 'p', 'p', 'p', 'e', 'e', 'p', 'e', 'p', 'e', 'e', 'e', 'e', 'p', 'e', 'p', 'e', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'e', 'e', 'p', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'e', 'e', 'p', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'p', 'p', 'p', 'e', 'p', 'e', 'e', 'p', 'p', 'p',